# Desafios idwall


## Resolução da Parte 1 do Desafio 2 - Crawlers

Web Scraping do Reddit, escrito em Python, utilizando as bibliotecas _request_ e _BeautifulSoup_.

#### Desafio:
Encontrar e listar as _threads_ com 5000 pontos ou mais no Reddit naquele momento.


### Resolução:

É possível realizar o scrapping via PRAW, um wrapper para o API do Reddit, o qual permite realizar scrapings dos subreddits, criar um bot, entre outras funcionalidades.

Segue um site que ensina realizar scrapping por esse método:
https://towardsdatascience.com/scraping-reddit-data-1c0af3040768

Mas aqui, com o intuito de demonstrar habilidades mais gerais, vamos realizar o scrapping utilizando os pacotes 'request' e 'BeautifulSoup'.

Para uma breve introdução sobre web scraping e aplicação destes pacotes ver:

https://www.scrapehero.com/a-beginners-guide-to-web-scraping-part-1-the-basics/

https://www.youtube.com/watch?v=ng2o98k983k&t=1428s


Dito isto, vamos começar pelo simples e buscar as _top threads_ dentro do subreddit 'r/AskReddit': https://www.reddit.com/r/AskReddit/top/?t=day

"Subreddits são como fóruns dentro do Reddit e as postagens são chamadas threads.

Para quem gosta de gatos, há o subreddit '/r/cats' com threads contendo fotos de gatos fofinhos. Para threads sobre o Brasil, vale a pena visitar '/r/brazil' ou ainda '/r/worldnews'. Um dos maiores subreddits é o '/r/AskReddit'."

### Abrindo a url e salvando o arquivo em html:

In [23]:
# Primeiramente, importam-se as bibliotecas necessárias para o scrapping:
import requests
from bs4 import BeautifulSoup

# E cria-se uma função para salvar e outra para
# abrir a página html, a fim de minimizar danos ao servidor:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

def open_html(path):
    with open(path, 'rb') as f:
        return f.read()

#### ATENÇÃO!!!

Recomenda-se não rodar o código da célula abaixo, sendo utilizado assim apenas para simples conferência.

Como o reddit possui um sistema automatizado que impede mais que um request a cada dois segundos é possível que o código abaixo gere um "erro", de forma a não ser capaz de obter o código html do site. Eu tentei procurar entender o porquê, mas não obtive uma resposta.

Mas caso queira rodar o código, é necessário tentar algumas vezes, caso não consiga de primeira, até conseguir obter o html.

In [24]:
# Para abrir localmente e trabalhar com o arquivo e não com vários requests:
html = open_html('askreddit_top_day')

soup = BeautifulSoup(html, 'lxml')

print(soup.prettify()[:1000])

<!DOCTYPE html>
<html lang="en">
 <head>
  <script>
   var __SUPPORTS_TIMING_API = typeof performance === 'object' && !!performance.mark && !! performance.measure && !!performance.getEntriesByType;
          function __perfMark(name) { __SUPPORTS_TIMING_API && performance.mark(name); };
          var __firstLoaded = false;
          function __markFirstPostVisible() {
            if (__firstLoaded) { return; }
            __firstLoaded = true;
            __perfMark("first_post_title_image_loaded");
          }
  </script>
  <script>
   __perfMark('head_tag_start');
  </script>
  <title>
   Ask Reddit...
  </title>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="origin-when-cross-origin" name="referrer"/>
  <style>
   /* http://meyerweb.com/eric/tools/css/reset/
    v2.0 | 20110126
    License: none (public domain)
  */

  html, body, div, span, applet, object, iframe,
  h1, h2, h3, h4, h5, h6, p, blockquote, pre,
  a, 

### Realizando o parsing no código reddit:

Primeiramente vamos analisar se o primeiro top thread é o desejado:

In [9]:
## html do bloco completo contendo todos os comentários
all_threads = soup.find('div', class_="rpBJOHq2PR60pnwJlUyP0")

## html da tag e classe do número de pontos de certa thread:
# <div class="_1rZYMD_4xY3gRcSS3p8ODO" style="color:#1A1A1B">22.8k</div>
pontos_thread = all_threads.find('div', class_="_1rZYMD_4xY3gRcSS3p8ODO")
pontos_primeira_thread = pontos_thread.text

## html da tag e classe do texto de uma certa thread
# <h3 class="_eYtD2XCVieq6emjKBH3m">Teachers of Reddit, what was the most obvious "teacher crush" someone had on you?</h3>
thread = all_threads.find('h3', class_="_eYtD2XCVieq6emjKBH3m")
texto_thread = thread.text

## html do link do texto
# <a data-click-id="body" class="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE" href="/r/AskReddit/comments/cyqtk2/teachers_of_reddit_what_was_the_most_obvious/"><div class="_2SdHzo12ISmrC8H86TgSCp _3wqmjmv3tb_k-PROt7qFZe " style="--posttitletextcolor:#444e59" theme="[object Object]"><h3 class="_eYtD2XCVieq6emjKBH3m">Teachers of Reddit, what was the most obvious "teacher crush" someone had on you?</h3></div></a>
link = all_threads.find('a', class_="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE")
referencia = link['href']
texto_link = f'https://www.reddit.com{referencia}'

In [10]:
# Resultado dos pontos e texto do primeiro top thread do dia:
print(pontos_primeira_thread)
print(texto_thread)
print(texto_link)

28.8k
Everyone has a scar on their body from something dumb, they did as a child. What's your story?
https://www.reddit.com/r/AskReddit/comments/cz2apy/everyone_has_a_scar_on_their_body_from_something/


Tudo certo até aqui, então vamos seguir com os próximos passos.

### Realizando um loop sobre todas as top threads desejadas:

In [11]:
table_threads = soup.find('div', class_="rpBJOHq2PR60pnwJlUyP0")

all_points_thread = table_threads.find_all('div', class_="_1rZYMD_4xY3gRcSS3p8ODO")
all_texts = table_threads.find_all('h3', class_="_eYtD2XCVieq6emjKBH3m")
all_links = table_threads.find_all('a', class_="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE")

extracted_points = []
for points in all_points_thread:
    point = points.text
    extracted_points.append(point)

extracted_texts = []
for threads in all_texts:
    thread = threads.text
    extracted_texts.append(thread)
    
extracted_links = []
for links in all_links:
    referencia = links['href']
    if referencia.startswith('http'):
        extracted_links.append(referencia)
    else:
        texto_link = f'https://www.reddit.com{referencia}'
        extracted_links.append(texto_link)
        

In [12]:
top_threads = []
for p, t, l in zip(extracted_points, extracted_texts, extracted_links):
    if len(p) > 1 and p[-1] == 'k': # condição para evitar promoted threads e threads com menos de 1000 pontos
        likes = p[0:-1]
        likes = int(float(likes)*1000)
        if likes >= 5000:
            subreddit = l.split('/')[4]
            s = f'/r/{subreddit}'
            record = {'pontuacao': p, 'subreddit': s, 'titulo thread': t, 'link para os comentarios': l}
            top_threads.append(record)
    
print(top_threads)

[{'pontuacao': '28.8k', 'subreddit': '/r/AskReddit', 'titulo thread': "Everyone has a scar on their body from something dumb, they did as a child. What's your story?", 'link para os comentarios': 'https://www.reddit.com/r/AskReddit/comments/cz2apy/everyone_has_a_scar_on_their_body_from_something/'}, {'pontuacao': '28.8k', 'subreddit': '/r/AskReddit', 'titulo thread': 'What is the scariest/creepiest/most disturbing thing you have ever encountered? [Serious]', 'link para os comentarios': 'https://www.reddit.com/r/AskReddit/comments/cyv6za/what_is_the_scariestcreepiestmost_disturbing/'}]


In [13]:
# Opcionalmente podemos salvar o resultando em um arquivo json para uso futuro

import json
with open('data.json', 'w') as outfile:
    json.dump(top_threads, outfile, indent=4)

### Realizando o scrapping na pagina principal

Agora vamos ir para uma página mais geral do site www.reddit.com, e realizar um novo scrape (Novamente, a célula abaixo foi convertida em Raw NBConvert, para evitar rodá-la acidentalmente):

In [57]:
# Código restante:

html = open_html('reddit_top_today')

soup = BeautifulSoup(html, 'lxml')

table_threads = soup.find('div', class_="rpBJOHq2PR60pnwJlUyP0")

all_points_thread = table_threads.find_all('div', class_="_1rZYMD_4xY3gRcSS3p8ODO")
all_texts = table_threads.find_all('h3', class_="_eYtD2XCVieq6emjKBH3m")
all_links = table_threads.find_all('a', class_="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE")

extracted_points = []
for points in all_points_thread:
    point = points.text
    extracted_points.append(point)

extracted_texts = []
for threads in all_texts:
    thread = threads.text
    extracted_texts.append(thread)
    
extracted_links = []
for links in all_links:
    referencia = links['href']
    if referencia.startswith('http'):
        extracted_links.append(referencia)
    else:
        texto_link = f'https://www.reddit.com{referencia}'
        extracted_links.append(texto_link)

top_threads = []
for p, t, l in zip(extracted_points, extracted_texts, extracted_links):
    if len(p) > 1 and p[-1] == 'k': # condição para evitar promoted threads e threads com menos de 1000 pontos
        likes = p[0:-1]
        likes = int(float(likes)*1000)
        if likes >= 5000:
            subreddit = l.split('/')[4]
            s = f'/r/{subreddit}'
            record = {'pontuacao': p, 'subreddit': s, 'titulo thread': t, 'link para os comentarios': l}
            top_threads.append(record)

for i in range(len(top_threads)):
    print(top_threads[i])
    print()

{'pontuacao': '121k', 'subreddit': '/r/aww', 'titulo thread': 'Scared cat gets saved by two French guys', 'link para os comentarios': 'https://www.reddit.com/r/aww/comments/cyv97r/scared_cat_gets_saved_by_two_french_guys/'}

{'pontuacao': '121k', 'subreddit': '/r/memes', 'titulo thread': 'The Area 51 raid is still happening right?', 'link para os comentarios': 'https://www.reddit.com/r/memes/comments/cz2i20/the_area_51_raid_is_still_happening_right/'}

{'pontuacao': '111k', 'subreddit': '/r/pics', 'titulo thread': 'In 1964, Ringo Starr snapped a photo of some high school students who skipped class to see the Beatles during their first trip to the US. The group had no idea the photo existed until Ringo published his book of photos. Nearly 50 years later, the group reunited and recreated the photo.', 'link para os comentarios': 'https://www.reddit.com/r/pics/comments/cyx1os/in_1964_ringo_starr_snapped_a_photo_of_some_high/'}

{'pontuacao': '111k', 'subreddit': '/r/aww', 'titulo thread': 

### Realizando o scrapping em uma lista de subreddits

Agora que entendemos como realizar o scrappping em um único link, o próximo passo seria realizar o scraping a partir de uma lista de subreddits separados por ponto-e-vírgula, e.g., "programming;dogs;brazil".

In [37]:
# input: programming;dogs;brazil
subreddits = str(input('Quais subreddits gostaria de acompanhar hoje? \n'))
subreddits_separated = subreddits.split(';')

print('\n', subreddits_separated)

Quais subreddits gostaria de acompanhar hoje? 
programming;dogs;brazil

 ['programming', 'dogs', 'brazil']


In [38]:
urls_subreddits = []
for i in range(len(subreddits_separated)):
    url_link = 'https://www.reddit.com/r/'+subreddits_separated[i]
    urls_subreddits.append(url_link)

print(urls_subreddits)

['https://www.reddit.com/r/programming', 'https://www.reddit.com/r/dogs', 'https://www.reddit.com/r/brazil']


Indo para a pasta principal, é possível ver que os arquivos foram salvos, mas por conta da proteção que o site reddit possui, os htmls salvos não correspondem aos desejados.

Por conta disto, foi necessário uma intervenção humana no looping para que fosse possível obter os htmls desejados.

Caso a célula acima funcionasse sem problemas, seria possível realizar o mesmo scrapping anterior utilizando um loop do código anterior, como mostrado abaixo: 

In [56]:
for i in subreddits_separated:
    
    html = open_html(f'reddit_{i}')

    soup = BeautifulSoup(html, 'lxml')

    table_threads = soup.find('div', class_="rpBJOHq2PR60pnwJlUyP0")

    all_points_thread = table_threads.find_all('div', class_="_1rZYMD_4xY3gRcSS3p8ODO")
    all_texts = table_threads.find_all('h3', class_="_eYtD2XCVieq6emjKBH3m")
    all_links = table_threads.find_all('a', class_="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE")

    extracted_points = []
    for points in all_points_thread:
        point = points.text
        extracted_points.append(point)

    extracted_texts = []
    for threads in all_texts:
        thread = threads.text
        extracted_texts.append(thread)

    extracted_links = []
    for links in all_links:
        referencia = links['href']
        if referencia.startswith('http'):
            extracted_links.append(referencia)
        else:
            texto_link = f'https://www.reddit.com{referencia}'
            extracted_links.append(texto_link)

    top_threads = []
    for p, t, l in zip(extracted_points, extracted_texts, extracted_links):
        if len(p) > 1 and p[-1] == 'k': # condição para evitar promoted threads e threads com menos de 1000 pontos
            likes = p[0:-1]
            likes = int(float(likes)*1000)
            if likes >= 5000:
                subreddit = l.split('/')[4]
                s = f'/r/{subreddit}'
                record = {'pontuacao': p, 'subreddit': s, 'titulo thread': t, 'link para os comentarios': l}
                top_threads.append(record)

    print(f'Top threads de /r/{i}: ', top_threads)
    print()

Top threads de /r/programming:  []

Top threads de /r/dogs:  []

Top threads de /r/brazil:  []



Nos casos acima, não houveram threads com mais de 5000 pontos. Além disto, foi possível rodar o código por conta da "intervenção" comentada anteriormente.